In [1]:
###########################################

In [2]:
############ Import libraries

In [3]:
from bs4 import BeautifulSoup as bs 
import pandas as pd
import re
from selenium import webdriver 
#from selenium.webdriver.common.keys import Keys
import time

ModuleNotFoundError: No module named 'bs4'

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
#####################################################
################ utility function
def return_string(string, start_after_word,stop_before_word):
    """return part of string between 2 words \
    (string, start_after_word,stop_befor_word)"""
    try:
        if start_after_word=="":
            start_address=0
        else:
            start_address=re.search(start_after_word,string).span()[1]
        #print(start_address)
        if stop_before_word=="":
            stop_address=len(string)
        else:
            stop_address=re.search(stop_before_word,string).span()[0]
        #print(stop_address)
        my_string=string[start_address:stop_address]
    except Exception as e:
        # handle error
        error=f"_exception_: {type(e).__name__},</br> _arguments_: {e.args}"
        print(error)
        my_string=None
    return my_string

In [ ]:
return_string('i love papa sdsds and mama sdsd .','mama','')

In [ ]:
# URL  of page for scraping
url='https://heb.com'

In [ ]:
driver = webdriver.Chrome('C://bin//chromedriver.exe')  
driver.get(url) 
time.sleep(5)

In [ ]:
#clear button
try:
    py_button=driver.find_element_by_xpath("//button[@aria-label='Reset']")
    py_button.click()
except:
    print('no text yet')

In [ ]:
# click on search field
py_button = driver.find_element_by_xpath("//input[@name='q']")
py_button.click()

In [ ]:
# search of Milk
search_item='milk'
py_button.send_keys(f'{search_item}')


In [ ]:
# press search button ( wait 5 min)
py_button=driver.find_element_by_xpath("//button[@aria-label='Search']")
py_button.click()
time.sleep(5)

In [ ]:

response = driver.page_source
soup=bs(response,'lxml')
soup

In [ ]:
result=soup.body.find_all('a',id=re.compile('product-'))
result[3]

In [ ]:
try_result=result[4]['aria-label'].strip()
try_result

In [ ]:
# getting features 
return_string(try_result,"Features:","")

In [ ]:
# getting coupon yes or no 
result[13].find('img',src=re.compile('coupon'))

In [ ]:

return_string(try_result,"","")

In [ ]:
# getting image URL 
result[0].find('img',src=re.compile('prd-small'))['src']

In [ ]:
result[0].find('span',class_=re.compile('responsive')).text.strip()

In [ ]:
result[0].find('span',class_=re.compile('uomSalePrice')).text.strip()


In [ ]:
result[0]['id']

In [ ]:
eval(result[0].find('script',type="application/ld+json").string)

In [ ]:
key_list=['id','name','brand','type','features','size','price','category','image','rating','json','uomSalePrice']

my_result=soup.body.find_all('a',id=re.compile('product-'))
scrap_df=pd.DataFrame()
i=0
for record in my_result:
    
    name=record.find('span',class_=re.compile('responsive')).text.strip()
    size=return_string(name,"Milk,","")
    json=eval(record.find('script',type="application/ld+json").string)
    id=json['id']
    brand=json['brand']
    category=json['category']
    price=json['price']
    search_string=record['aria-label'].strip()
    features=return_string(search_string,"Features:","")
  #  rating=return_string(search_string,"Rated","stars")
    uomSalePrice=record.find('span',class_=re.compile('uomSalePrice')).text.strip()
    image=record.find('img',src=re.compile('prd-small'))['src']
    if record.find('img',src=re.compile('coupon'))==None:
        coupon=0
    else:
        coupon=1
    print(i)
    print (f'{name} \n{size}\n\n{id}\n{brand}\n{category}\n{price}\n{features}\n{coupon}\n{uomSalePrice}\n{image}')
    print('___________________')
    i+=1
    scrap_df = scrap_df.append({'id':id,
                                'name': name,
                                'brand':brand,
                                'size':size,
                                'category':category,
                                'price':price,
                                'features':features,
                                'coupon':coupon,
                                'uomSalePrice':uomSalePrice,
                                'image':image,
                               }, ignore_index=True)
    
    

In [ ]:
scrap_df

In [ ]:
scrap_df[scrap_df.isna().any(axis=1)]



In [ ]:
re.compile("n*")



In [ ]:
scrap_df.to_csv('heb.csv')

In [ ]:
driver.close()